Code adapted from Tensorflow Object Detection Framework
https://github.com/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb
Tensorflow Object Detection Detector

In [1]:
import argparse
import sys
import time

import cv2
import numpy as np
import tensorflow as tf

import track.seattracker as seattrack
import track.tracker as track
import scripts.detector as od

ImportError: Traceback (most recent call last):
  File "c:\users\zhiqi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\zhiqi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\zhiqi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 17, in swig_import_helper
    return importlib.import_module(mname)
  File "c:\users\zhiqi\appdata\local\programs\python\python36\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
ImportError: DLL load failed: The paging file is too small for this operation to complete.


Failed to load the native TensorFlow runtime.

See https://www.t

# Function to draw tracked person and seats

In [3]:
def drawTrackedFace(imgDisplay):
    for fid in tracker.faceTrackers.keys():
        tracked_position = tracker.faceTrackers[fid].get_position()
        t_x = int(tracked_position.left())
        t_y = int(tracked_position.top())
        t_w = int(tracked_position.width())
        t_h = int(tracked_position.height())

        timestayed = int(tracker.getTimeInSecond(fid))
        text = 'P{} '.format(fid) + str(timestayed) + 's'
        rectColor = (0, 255, 0)

        textSize = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
        textX = int(t_x + t_w / 2 - (textSize[0]) / 2)
        textY = int(t_y)
        textLoc = (textX, textY)

        cv2.rectangle(imgDisplay, (t_x, t_y),
                      (t_x + t_w, t_y + t_h),
                      rectColor, 1)

        cv2.putText(imgDisplay, text, textLoc,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 0, 0), 1)


def drawTrackedSeat(imgDisplay):
    for fid in seattracker.faceTrackers.keys():
        tracked_position = seattracker.faceTrackers[fid].get_position()
        t_x = int(tracked_position.left())
        t_y = int(tracked_position.top())
        t_w = int(tracked_position.width())
        t_h = int(tracked_position.height())

        text = 'S{}'.format(fid)
        occupiedbool = seattracker.occupied[fid]
        if occupiedbool:
            rectColor = (255, 0, 0)
        else:
            rectColor = (0, 0, 255)

        textSize = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
        textX = int(t_x + t_w / 2 - (textSize[0]) / 2)
        textY = int(t_y)
        textLoc = (textX, textY)

        cv2.rectangle(imgDisplay, (t_x, t_y),
                      (t_x + t_w, t_y + t_h),
                      rectColor, 1)

        cv2.putText(imgDisplay, text, textLoc,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (255, 255, 255), 1)


# Function to parse arguments

In [5]:

def check_arg(args=None):
    parser = argparse.ArgumentParser(description='Script for counting person in zone and free/occupied seat detection')
    parser.add_argument('-m', '--model',
                        help='Tensorflow object detection model path',
                        required=True,
                        default='model/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb')
    parser.add_argument('-i', '--input',
                        help='Input video filename',
                        required=True)
    parser.add_argument('-o', '--output',
                        help='Filename for output video',
                        default='output.avi')
    parser.add_argument('-f', '--frame_interval',
                        help='Amount of frame interval between frame processing',
                        default=5)
    parser.add_argument('-p', '--roi_points',
                        help='Points for ROI in format of (x1,y1,x2,y2)',
                        required=True)
    parser.add_argument('-pt', '--people_threshold',
                        help='Threshold value for people detection',
                        default=0.9)
    parser.add_argument('-st', '--seat_threshold',
                        help='Threshold value for seat detection',
                        default=0.9)
    results = parser.parse_args(args)
    return (results.model,
            results.input,
            results.output,
            results.frame_interval,
            results.roi_points,
            results.seat_threshold,
            results.people_threshold)


# Main function to process video

In [ ]:
# getting variables from sys arguments
model_path, input, output, frame_interval, points, seatthreshold, threshold = check_arg(sys.argv[1:])
frame_interval = int(frame_interval)
seatthreshold = float(seatthreshold)
threshold = float(threshold)
points = eval(points)
p1 = (points[0], points[1])
p2 = (points[2], points[3])

In [ ]:
# manually setting variables
model_path = 'model/frozen_inference_graph.pb'
input = 'videos/A0051.mov'
output = 'output/sample_output.avi'
frame_interval = 5
threshold = 0.8
seatthreshold = 0.8
pt1 = (0, 0)
pt2 = (500, 500)

In [ ]:
# initiaizing variables and object detector and cv2 video capture
id = 0
seat = 0
frame_count = 0
odapi = od.DetectorAPI(path_to_ckpt=model_path)
cap = cv2.VideoCapture(input)
flag, frame = cap.read()
assert flag == True
# getting video infomation
tracker.videoFrameSize = frame.shape
seattracker.videoFrameSize = frame.shape
fps = cap.get(cv2.CAP_PROP_FPS)
tracker.fps = fps

fourcc = cv2.VideoWriter_fourcc(*'XVID')
# Define VideoWrite object
# cv2.VideoWrite('arg1',arg2,arg3,(width,heigh))
# arg1:output file name
# arg2:Specify Fourcc code
# arg3: frames per seconds
# FourCC is a 4-byte code used to specify video codec
out = cv2.VideoWriter(output, fourcc, 25.0, (768, 432))

while True:
    r, img = cap.read()
    # if there are still frames to process
    if r:
        if frame_count % (frame_interval * 5) == 0:
            # remove any duplicate tracker that overlaps each other.
            tracker.removeDuplicate()
        
        # update and/or delete all trackers
        tracker.deleteTrack(img)
        seattracker.deleteTrack(img)
        # check each seat whether it is occupied or not based on euclidean distance of seat from each person
        seattracker.checkOccupied(tracker)

        if frame_count % frame_interval == 0:
            # cropping ROI from image
            roi = img[p1[1]:p2[1], p1[0]:p2[0]]
            boxes, scores, classes, num = odapi.processFrame(roi)
            # Visualization of the results of a detection.
            for i in range(len(boxes)):
                box = boxes[i]
                # adding back the cropped region
                x1 = box[1] + p1[0]
                y1 = box[0] + p1[1]
                x2 = box[3] + p1[0]
                y2 = box[2] + p1[1]
                # Class 1 represents human
                if classes[i] == 1 and scores[i] >= threshold:

                    matchedID = tracker.getMatchId(img, (x1, y1, x2, y2))
                    if matchedID is None:
                        id += 1
                        tracker.createTrack(img, (x1, y1, x2, y2), str(id), scores[i])
                    # cv2.rectangle(img,(box[1],box[0]),(box[3],box[2]),(255,0,0),2)
                # Class 2 represents seat in self trained model or use Class 62 if coco datasets-trained model is used
                elif (classes[i] == 2) and scores[i] >= seatthreshold:
                    matchedID = seattracker.getMatchId(img, (x1, y1, x2, y2))
                    if matchedID is None:
                        seat += 1
                        seattracker.createTrack(img, (x1, y1, x2, y2), str(seat), scores[i])
        # draw tracked seat and person and get person count
        drawTrackedSeat(img)
        drawTrackedFace(img)
        number = int(len(tracker.faceTrackers))
        cv2.rectangle(img, p1, p2, (0, 255, 0), 2)
        cv2.putText(img, 'People: ' + str(number), (0, 25),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 255, 0), 2)
        occupied, unoccupied = seattracker.getAmount()
        cv2.putText(img, 'Occupied Seat: {}'.format(str(occupied)), (0, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (255, 0, 0), 2)
        cv2.putText(img, 'Free Seat: {}'.format(str(unoccupied)), (0, 75),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 2)
        out.write(img)
        frame_count += 1
        # show frame in cv2
        # cv2.imshow("preview", img)
        key = cv2.waitKey(1)
        if key & 0xFF == ord('q'):
            break
    else:
        raise RuntimeError('No more frame')